In [1]:
# Funciones útiles
def convert(ruta): 
    s = [str(i) for i in ruta] 
    ruta_c = "-".join(s)
    return(ruta_c) 

In [2]:
import pandas as pd
#Cargamos el csv con los datos
df = pd.read_csv("C:/Documents/Proyecto final/base.csv", sep = ',',encoding='latin-1')

In [3]:
#Creación de función que genera el diccionario necesario para ingresar al algoritmo
def diccionario_lugares(df,fv):
    """
    Función para generar un diccinario con los lugares que visita una fuerza de venta.
    Args:
        df (dataFrame): DataFrame con la base de datos
        fv (int): Fuerza de venta del que queremos generar el diccionario.
    Returns:
        lugares(diccionario): incluye los puntos que debe visitar con sus coordenas. 
    """
    df2 = df[(df.fza_ventas == fv)]
    df2['lugar'] = range(1, len(df2) + 1)
    df2 = df2.append({'lugar' : 'base' , 'lat_destino' : df2.iloc[0]['lat_origen'],'lon_destino': df2.iloc[0]['lon_origen'] } , ignore_index=True)
    df3 = df2[['lugar','lat_destino','lon_destino']].copy()
    lugares = {a:(b,c)for a, b, c in df3.values}
    return lugares

In [12]:
lugares = diccionario_lugares(df,94459)
#Podemos probar con estas fuerza de venta:
# 94459- 6 lugares, Nuevo león
# 37831- 30 lugares, Jalisco
#  14738- 58 lugares, Ciudad de México

C:\Users\anabc\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [13]:
lugares

{1: (25.671909600000003, -100.1774657),
 2: (25.664719, -100.1628501),
 3: (25.672287100000002, -100.1651263),
 4: (25.678590399999997, -100.16507790000001),
 5: (25.68714417, -100.1897223),
 6: (25.6778829, -100.1665256),
 'base': (25.6626, -100.204)}

In [14]:
# -*- coding: utf-8 -*-
from __future__ import print_function
import math
import random
from simanneal import Annealer


def distance(a, b):
    """Calculates distance between two latitude-longitude coordinates."""
    R = 6371  # radius of Earth (miles)
    lat1, lon1 = math.radians(a[0]), math.radians(a[1])
    lat2, lon2 = math.radians(b[0]), math.radians(b[1])
    return math.acos(math.sin(lat1) * math.sin(lat2) +
                     math.cos(lat1) * math.cos(lat2) * math.cos(lon1 - lon2)) * R


class TravellingSalesmanProblem(Annealer):

    """Test annealer with a travelling salesman problem.
    """

    # pass extra data (the distance matrix) into the constructor
    def __init__(self, state, distance_matrix):
        self.distance_matrix = distance_matrix
        super(TravellingSalesmanProblem, self).__init__(state)  # important!

    def move(self):
        """Swaps two cities in the route."""
        # no efficiency gain, just proof of concept
        # demonstrates returning the delta energy (optional)
        initial_energy = self.energy()

        a = random.randint(0, len(self.state) - 1)
        b = random.randint(0, len(self.state) - 1)
        self.state[a], self.state[b] = self.state[b], self.state[a]

        return self.energy() - initial_energy

    def energy(self):
        """Calculates the length of the route."""
        e = 0
        for i in range(len(self.state)):
            e += self.distance_matrix[self.state[i-1]][self.state[i]]
        return e

In [15]:
rutas = pd.DataFrame(index=range(10),columns=['km', 'ruta'])

In [16]:
# initial state, a randomly-ordered itinerary
for i in range(10):
    init_state = list(lugares.keys())
    random.shuffle(init_state)

    # create a distance matrix
    distance_matrix = {}
    for ka, va in lugares.items():
        distance_matrix[ka] = {}
        for kb, vb in lugares.items():
            if kb == ka:
                distance_matrix[ka][kb] = 0.0
            else:
                distance_matrix[ka][kb] = distance(va, vb)

    tsp = TravellingSalesmanProblem(init_state, distance_matrix)
    tsp.set_schedule(tsp.auto(minutes=0.2))
    # since our state is just a list, slice is the fastest way to copy
    tsp.copy_strategy = "slice"
    state, e = tsp.anneal()

    while state[0] != 'base':
        state = state[1:] + state[:1]  # rotate NYC to start

        print()
        print("%i mile route:" % e)
        print("\t", state)

        rutas.km[i] = e
        ruta = state
        ruta = convert(ruta)
        rutas.ruta[i] = ruta

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00580         11.88    16.00%     0.00%     0:00:02    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00580         11.88    14.22%     0.03%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     3.00000         16.85    80.45%    32.65%     0:00:00     0:00:01


11 mile route:
	 ['base', 5, 6, 4, 3, 2, 1]


     0.01500         12.32    13.60%     0.00%     0:00:02     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01500         11.88    15.20%     0.14%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
    32.00000         18.27    98.60%    41.50%     0:00:00     0:00:01


11 mile route:
	 [2, 3, 4, 6, 5, 'base', 1]

11 mile route:
	 [3, 4, 6, 5, 'base', 1, 2]

11 mile route:
	 [4, 6, 5, 'base', 1, 2, 3]

11 mile route:
	 [6, 5, 'base', 1, 2, 3, 4]

11 mile route:
	 [5, 'base', 1, 2, 3, 4, 6]

11 mile route:
	 ['base', 1, 2, 3, 4, 6, 5]


     0.00870         11.88    14.90%     0.00%     0:00:02     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00870         11.88    14.42%     0.02%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.92000         13.09    50.55%    17.65%     0:00:00     0:00:01

...


11 mile route:
	 ['base', 1, 2, 3, 4, 6, 5]


     0.00670         11.88    14.40%     0.00%     0:00:02    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00670         11.88    15.07%     0.00%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.80000         12.37    46.30%    16.65%     0:00:01     0:00:00


11 mile route:
	 [3, 4, 6, 5, 'base', 1, 2]

11 mile route:
	 [4, 6, 5, 'base', 1, 2, 3]

11 mile route:
	 [6, 5, 'base', 1, 2, 3, 4]

11 mile route:
	 [5, 'base', 1, 2, 3, 4, 6]

11 mile route:
	 ['base', 1, 2, 3, 4, 6, 5]


     0.00870         11.88    12.85%     0.00%     0:00:01     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00870         11.88    14.72%     0.04%     0:00:06     0:00:00


11 mile route:
	 ['base', 1, 2, 3, 4, 6, 5]


In [17]:
rutas

,km,ruta
0,11.8843,base-5-6-4-3-2-1
1,11.8843,base-1-2-3-4-6-5
2,11.8843,base-5-6-4-3-2-1
3,NaN,NaN
4,11.8843,base-1-2-3-4-6-5
5,11.8843,base-5-6-4-3-2-1
6,11.8843,base-1-2-3-4-6-5
7,11.8843,base-1-2-3-4-6-5
8,11.8843,base-1-2-3-4-6-5
9,11.8843,base-1-2-3-4-6-5
